In [2]:
import random
import pandas as pd
import string
from datasets import Dataset

In [4]:
def generate_random_dataset():
    ds = dict()
    exemplar_snippet = "def hello(): print('Hello World!')"
    ds["input"] = []
    ds["output"] = []
    for i in range(100000):
        random_string = ''.join(random.choices(string.ascii_lowercase, k = 5))
        ds["input"].append(random_string)
        ds["output"].append(exemplar_snippet)
    return pd.DataFrame.from_dict(ds)

In [23]:
df = generate_random_dataset()
ds = Dataset.from_pandas(df)

In [24]:
df.head(3)

,input,output
0,ismha,def hello(): print('Hello World!')
1,mxrap,def hello(): print('Hello World!')
2,hdgmm,def hello(): print('Hello World!')


In [29]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")
prefix = "translate bad code to good code: "
def preprocess_function(examples):
    inputs = [prefix + example for example in examples["input"]]
    outputs = [example for example in examples["output"]]
    model_inputs = tokenizer(inputs, text_target=outputs, max_length=128, truncation=True)
    return model_inputs
    

/Users/michaelnath/.virtualenvs/224n_final_proj/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [30]:
tokenized_code = ds.map(preprocess_function, batched=True)

100%|██████████| 100/100 [00:04<00:00, 23.59ba/s]


In [39]:
tokenized_code = tokenized_code.train_test_split(test_size=0.2)

AttributeError: 'DatasetDict' object has no attribute 'train_test_split'

In [43]:
print(tokenized_code["test"])

Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 20000
})


In [34]:
import evaluate

sacrebleu = evaluate.load("sacrebleu")

In [44]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [32]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

In [45]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_code_code_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_code["train"],
    eval_dataset=tokenized_code["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA devices.